In [17]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from openai import OpenAI
from dotenv import load_dotenv, dotenv_values
load_dotenv()

client = OpenAI()

Scrape all links on the main doc page, then grab the content from each link as well as the sub-links within them.

In [26]:
import os
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

def scrape_and_save(url, directory='results', depth=1):
    # Send a GET request to the webpage
    response = requests.get(url)

    # If the GET request is successful, the status code will be 200
    if response.status_code == 200:
        # Get the content of the response
        content = response.content

        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(content, 'html.parser')

        # Find the main content
        main_content = soup.find('div', id="mc-main-content")

        # Save the main content to a file
        if main_content:
            with open(os.path.join(directory, f'{urlparse(url).path.split("/")[-1]}.txt'), 'w', encoding='utf-8') as f:
                f.write(main_content.get_text())

        # Initialize links as an empty list
        links = []

        # If depth is 1, find normal href links
        if depth == 2:
            links = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if not a['href'].startswith(('javascript:', 'tel:'))]
        # If depth is greater than 1, look for the sub-menu
        elif depth > 0:
            # Find the sub-menu element
            sub_menu = soup.find(class_='guideTOC')

            # If sub-menu is found, find all anchor tags within the sub-menu and extract href attributes
            if sub_menu:
                links = [urljoin(url, a['href']) for a in sub_menu.find_all('a', href=True) if not a['href'].startswith(('javascript:', 'tel:'))]

        # Loop through the links
        for link in links:
            scrape_and_save(link, directory, depth - 1)  # Recursively scrape and save content with reduced depth

    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)


# URL of the page you want to scrape
base_url = "https://help.relativity.com/RelativityOne/Content/index.htm"

# Create a new directory for the results
os.makedirs('results', exist_ok=True)

# Scrape and save content with depth limited to two levels
scrape_and_save(base_url, depth=2)

Failed to retrieve the webpage. Status code: 401
Failed to retrieve the webpage. Status code: 404


Do the same as above, but start on the Capabilities page.

In [41]:
import os
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

def scrape_and_save(url, directory='results_pidetect', depth=1):
    # Send a GET request to the webpage
    response = requests.get(url)

    # If the GET request is successful, the status code will be 200
    if response.status_code == 200:
        # Get the content of the response
        content = response.content

        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(content, 'html.parser')

        # Find the main content
        main_content = soup.find('div', id="mc-main-content")

        # Save the main content to a file
        if main_content:
            with open(os.path.join(directory, f'{urlparse(url).path.split("/")[-1]}.txt'), 'w', encoding='utf-8') as f:
                f.write(main_content.get_text())

        # Initialize links as an empty list
        links = []

        # If depth is 1, find normal href links
        if depth == 2:
            links = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if not a['href'].startswith(('javascript:', 'tel:'))]
        # If depth is greater than 1, look for the sub-menu
        elif depth > 0:
            # Find the sub-menu element
            sub_menu = soup.find(class_='guideTOC')

            # If sub-menu is found, find all anchor tags within the sub-menu and extract href attributes
            if sub_menu:
                links = [urljoin(url, a['href']) for a in sub_menu.find_all('a', href=True) if not a['href'].startswith(('javascript:', 'tel:'))]

        # Loop through the links
        for link in links:
            scrape_and_save(link, directory, depth - 1)  # Recursively scrape and save content with reduced depth

    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)


# URL of the page you want to scrape
base_url = "https://help.relativity.com/RelativityOne/Content/PI_Detect/PI_Detect.htm"

# Create a new directory for the results
os.makedirs('results_pidetect', exist_ok=True)

# Scrape and save content with depth limited to two levels
scrape_and_save(base_url, depth=2)

In [21]:
import os
from urllib.parse import urlparse

# URL of the page you want to scrape
base_url = "https://help.relativity.com/RelativityOne/Content/Relativity/aiR_for_Review/aiR_for_Review.htm"

# Create a new directory for the results
os.makedirs('results', exist_ok=True)

# Send a GET request to the webpage
response = requests.get(base_url)

if response.status_code == 200:
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    paragraphs = soup.find_all('div', id="mc-main-content")
    with open(os.path.join('results', 'aiR_for_Review.htm.txt'), 'w', encoding='utf-8') as f:
        # Loop through the paragraphs, get the text of each one, and write it to the file
        for paragraph in paragraphs:
            f.write(paragraph.get_text())
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

# If the GET request is successful, the status code will be 200
if response.status_code == 200:
    
    # Get the content of the response
    content = response.content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(content, 'html.parser')

    # Find the sub-menu element
    sub_menu = soup.find(class_='guideTOC')

    # Find all anchor tags within the sub-menu and extract href attributes
    links = [urljoin(base_url, a['href']) for a in sub_menu.find_all('a', href=True)]

    # Loop through the links
    for link in links:
        # Send a GET request to the child page
        response = requests.get(link)

        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the content of the response
            content = response.content

            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(content, 'html.parser')

            # Find the specified div tags
            paragraphs = soup.find_all('div', id="mc-main-content")

            # Get the page name from the link
            page_name = urlparse(link).path.split('/')[-1]

            # Open a file with the page name in the results directory
            with open(os.path.join('results', f'{page_name}.txt'), 'w', encoding='utf-8') as f:
                # Loop through the paragraphs, get the text of each one, and write it to the file
                for paragraph in paragraphs:
                    f.write(paragraph.get_text())
        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

KeyboardInterrupt: 

In [7]:
# URL of the page you want to scrape
base_url = "https://help.relativity.com/RelativityOne/Content/Relativity/aiR_for_Review/aiR_for_Review.htm"
llm_input = ''

# Send a GET request to the webpage
response = requests.get(base_url)

# Include the base_url
if response.status_code == 200:
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    paragraphs = soup.find_all('div', id="mc-main-content")
    for paragraph in paragraphs:
        llm_input += paragraph.get_text()
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

# If the GET request is successful, the status code will be 200
if response.status_code == 200:
    # Get the content of the response
    content = response.content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(content, 'html.parser')

    # Find the sub-menu element
    sub_menu = soup.find(class_='guideTOC')

    # Find all anchor tags within the sub-menu and extract href attributes
    links = [urljoin(base_url, a['href']) for a in sub_menu.find_all('a', href=True)]

    # Loop through the links
    for link in links:
        # Send a GET request to the child page
        response = requests.get(link)

        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the content of the response
            content = response.content

            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(content, 'html.parser')

            # Find the specified div tags
            paragraphs = soup.find_all('div', id="mc-main-content")

            # Loop through the paragraphs, get the text of each one, and print it
            for paragraph in paragraphs:
                llm_input += paragraph.get_text()
        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

print(llm_input)


aiR for Review
aiR for Review harnesses the power of large language models, or LLMs, to review documents. aiR for Review goes far beyond existing classifiers by using generative AI to both predict coding decisions and to support those predictions with descriptive text and document excerpts which explain the decisions.
Some benefits of aiR for Review include:


Highly efficient, low-cost document analysis


Quick discovery of important issues and criteria


Consistent, cohesive analysis across all documents


Note: aiR for Review is currently in limited release. For information about the general release, contact your account representative.
See these related pages:

Running aiR for Review

Monitoring aiR for Review jobs

aiR for Review results

aiR for Review security permissions


aiR for Review overview
aiR for Review uses generative AI to simulate the actions of a human reviewer, finding and describing relevant documents according to the review instructions that you provide. It iden

In [16]:
import os
from urllib.parse import urlparse

# URL of the page you want to scrape
base_url = "https://help.relativity.com/RelativityOne/Content/Relativity/aiR_for_Review/aiR_for_Review.htm"

# Send a GET request to the webpage
response = requests.get(base_url)

if response.status_code == 200:
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    paragraphs = soup.find_all('div', id="mc-main-content")
    with open(os.path.join('results', f'aiR_for_Review.htm.txt'), 'w', encoding='utf-8') as f:
        # Loop through the paragraphs, get the text of each one, and write it to the file
        for paragraph in paragraphs:
            f.write(paragraph.get_text())
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

# If the GET request is successful, the status code will be 200
if response.status_code == 200:
    
    # Get the content of the response
    content = response.content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(content, 'html.parser')

    # Find the sub-menu element
    sub_menu = soup.find(class_='guideTOC')

    # Find all anchor tags within the sub-menu and extract href attributes
    links = [urljoin(base_url, a['href']) for a in sub_menu.find_all('a', href=True)]

    # Loop through the links
    for link in links:
        # Send a GET request to the child page
        response = requests.get(link)

        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the content of the response
            content = response.content

            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(content, 'html.parser')

            # Find the specified div tags
            paragraphs = soup.find_all('div', id="mc-main-content")

            # Get the page name from the link
            page_name = urlparse(link).path.split('/')[-1]

            # Open a file with the page name
            with open(os.path.join('results', f'{page_name}.txt'), 'w', encoding='utf-8') as f:
                # Loop through the paragraphs, get the text of each one, and write it to the file
                for paragraph in paragraphs:
                    f.write(paragraph.get_text())
        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

In [7]:
#Pass the documentation text to GPT-3.5 Turbo and ask a question.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are a teacher helping me understand the following documentation on the aiR for Review product: {llm_input}"},
    {"role": "user", "content": f"What permissions are needed to run aiR for Review?"}
  ]
)

print(completion.choices[0].message.content)

To run aiR for Review, you need the following permissions:

1. **aiR for Review Profile - View, Edit, Add**: This permission allows you to interact with aiR for Review, including setting up, editing, and adding new profiles.

2. **Mass Operations - aiR for Review**: This permission enables you to run mass operations related to aiR for Review, such as analyzing documents, setting up prompt criteria, and submitting jobs for analysis.

Make sure you also belong to at least one user group other than the Workspace Admin Group for the system to recognize and authorize you to run aiR for Review.


In [8]:
assistant = client.beta.assistants.create(
  name="Relativity Tutor",
  instructions="You are a personal tutor for the RelativityOne application. Review documentation and answer questions to help me learn.",
  model="gpt-4-turbo",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=f"I have a question about {llm_input}. What permissions are needed to run aiR for Review?"
)

from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Kelly. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > To run the aiR for Review mass action, the necessary permissions are as follows:

1. **Object Security:**
   - aiR for Review Profile: View, Edit, Add permissions are needed.
   
2. **Other Settings:**
   - Mass Operations: Permission specifically for aiR for Review must be granted.

Additionally, you must be a member of at least one user group other than the Workspace Admin Group to perform actions related to aiR for Review. These permissions ensure authorized access for configuration and execution of reviews using the aiR for Review system within your workspace.

Do it without streaming

In [9]:
assistant = client.beta.assistants.create(
  name="Relativity Tutor",
  instructions="You are a personal tutor for the RelativityOne application. Review documentation and answer questions to help me learn.",
  model="gpt-4-turbo",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=f"I have a question about {llm_input}. What permissions are needed to run aiR for Review?"
)

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Kelly. The user has a premium account."
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_y43WaQNzHmVYjipTDbSWqlNo', assistant_id='asst_EnPQXTGuBBXUbQd6AuHc2zO4', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='To run aiR for Review, you will need specific permissions set up within your system. Here are the required permissions for running the aiR for Review mass action, based on the information provided:\n\n### Object Security\n- **aiR for Review Profile:** You need to have View, Edit, and Add permissions.\n\n### Other Settings\n- **Mass Operations:** Access to aiR for Review within the Mass Operations settings is required.\n\nAdditionally, to successfully operate aiR for Review, you must be part of at least one user group other than the Workspace Admin Group. This ensures that you have the appropriate level of access to handle sensitive data and utilize the aiR for Review tools effectively within your workspace.'), type='text')], created_at=1715285748, file_ids=[], incomplete_at=None, incomplete_d